In [1]:
#-*- coding: utf-8 -*-

### 데이터를 생성할 때 근소한 차이가 있을 수 있으니, 
### 보고서와 같은 결과를 얻기 위해서는 별도로 첨부한 osan_dataset.geojson을 사용하셔야 합니다.
#### 'osan.geojson' 대신 'osan_dataset.geojson'(별도 첨부)
#### 'syn.geojson' 대신 'syn_dataset.geojson'(별도 첨부)

# 데이터 및 패키지 불러오기

In [3]:
from geoband.API import *

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
import geopandas as gpd
import os
import pydeck as pdk
import shapefile

In [5]:
#변수선택법

import time
import itertools

In [6]:
#회귀분석

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.model_selection import train_test_split

In [7]:
#공간회귀분석

import pysal
import libpysal
from libpysal.weights import Queen, Rook, KNN, DistanceBand, Kernel
from esda.getisord import G
from esda.moran import Moran
from spreg import OLS, ML_Lag, ML_Error, GM_Lag, GM_Error, TSLS

In [8]:
#정규성검정

from scipy.stats import shapiro

In [9]:
#다중공선성 체크

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [10]:
osan = gpd.read_file('osan_dataset.geojson')

In [11]:
osan = sm.add_constant(osan, has_constant='add')

# 모델링

## 데이터셋 정의 및 다중공선성 확인

In [12]:
#데이터셋 정의

X = osan[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '횡단보도_포함면적', '신호등', 
         '단속횟수_횡단보도위', '단속횟수', 'traffic_sum', 'traffic_mean', 'freq_sum', 'freq_mean', 'time_sum', 'time_mean', 
         '초등학교_100m', '초등학교_300m', '초등학교_500m', '초등학교_가중치', '유치원_100m', '유치원_300m', '유치원_500m', '유치원_가중치',
         '유동인구', 'cctv', '버스정류장', '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'building', 'house', 'office', 
         'region_type', 'float_type', 'region_type_HS', 'region_type_OF', 'region_type_OTH', 'float_type_H', 'float_type_M', 'float_type_L', 
         'gid', 'geometry']]

y = osan[['사고횟수', 'gid', 'geometry']]

### 독립변수 종속변수 생성


In [13]:
#초등학교_100m, 유치원_100m
x_reg_1 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_100m', '유치원_100m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_1 = y[['사고횟수']]

#초등학교_100m, 유치원_300m
x_reg_2 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_100m', '유치원_300m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_2 = y[['사고횟수']]

#초등학교_100m, 유치원_500m
x_reg_3 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_100m', '유치원_500m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_3 = y[['사고횟수']]

#초등학교_100m, 유치원_가중치
x_reg_4 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_100m', '유치원_가중치', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_4 = y[['사고횟수']]

#초등학교_300m, 유치원_100m
x_reg_5 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_300m', '유치원_100m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_5 = y[['사고횟수']]

#초등학교_300m, 유치원_300m
x_reg_6 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_300m', '유치원_300m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_6 = y[['사고횟수']]

#초등학교_300m, 유치원_500m
x_reg_7 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_300m', '유치원_500m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_7 = y[['사고횟수']]

#초등학교_300m, 유치원_가중치
x_reg_8 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_300m', '유치원_가중치', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_8 = y[['사고횟수']]

#초등학교_500m, 유치원_100m
x_reg_9 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_500m', '유치원_100m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_9 = y[['사고횟수']]

#초등학교_500m, 유치원_300m
x_reg_10 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_500m', '유치원_300m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_10 = y[['사고횟수']]

#초등학교_500m, 유치원_500m
x_reg_11 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_500m', '유치원_500m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_11 = y[['사고횟수']]

#초등학교_500m, 유치원_가중치
x_reg_12 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_500m', '유치원_가중치', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_12 = y[['사고횟수']]

#초등학교_가중치, 유치원_100m
x_reg_13 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_가중치', '유치원_100m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_13 = y[['사고횟수']]

#초등학교_가중치, 유치원_300m
x_reg_14 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_가중치', '유치원_300m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_14 = y[['사고횟수']]

#초등학교_가중치, 유치원_500m
x_reg_15 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_가중치', '유치원_500m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_15 = y[['사고횟수']]

#초등학교_가중치, 유치원_가중치
x_reg_16 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', 'time_sum', '초등학교_가중치', '유치원_가중치', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_16 = y[['사고횟수']]

### 1차 다중공선성 확인

In [14]:
#초등학교_100m, 유치원_100m vif
vif1 = pd.DataFrame()
vif1['VIF Factor'] = [variance_inflation_factor(x_reg_1.values, i) for i in range(x_reg_1.shape[1])]
vif1['features'] = x_reg_1.columns
vif1

,VIF Factor,features
0,3.896033,const
1,1.269417,단속카메라
2,1.646612,도로안내표지
3,1.216494,과속방지턱
4,1.551880,스쿨존
5,2.527156,횡단보도_전체면적
6,2.290877,신호등
7,1.513835,단속횟수_횡단보도위
8,1.749845,단속횟수
9,2.280357,traffic_sum


In [15]:
#초등학교_100m, 유치원_300m vif
vif2 = pd.DataFrame()
vif2['VIF Factor'] = [variance_inflation_factor(x_reg_2.values, i) for i in range(x_reg_2.shape[1])]
vif2['features'] = x_reg_2.columns
vif2

,VIF Factor,features
0,3.899562,const
1,1.270727,단속카메라
2,1.645938,도로안내표지
3,1.220436,과속방지턱
4,2.013650,스쿨존
5,2.536244,횡단보도_전체면적
6,2.290648,신호등
7,1.513010,단속횟수_횡단보도위
8,1.749965,단속횟수
9,2.278973,traffic_sum


In [16]:
#초등학교_100m, 유치원_500m vif
vif3 = pd.DataFrame()
vif3['VIF Factor'] = [variance_inflation_factor(x_reg_3.values, i) for i in range(x_reg_3.shape[1])]
vif3['features'] = x_reg_3.columns
vif3

,VIF Factor,features
0,3.977760,const
1,1.269054,단속카메라
2,1.646126,도로안내표지
3,1.215515,과속방지턱
4,1.828041,스쿨존
5,2.529725,횡단보도_전체면적
6,2.290356,신호등
7,1.512963,단속횟수_횡단보도위
8,1.749464,단속횟수
9,2.282607,traffic_sum


In [17]:
#초등학교_100m, 유치원_가중치 vif
vif4 = pd.DataFrame()
vif4['VIF Factor'] = [variance_inflation_factor(x_reg_4.values, i) for i in range(x_reg_4.shape[1])]
vif4['features'] = x_reg_4.columns
vif4

,VIF Factor,features
0,3.896033,const
1,1.269417,단속카메라
2,1.646612,도로안내표지
3,1.216494,과속방지턱
4,1.551880,스쿨존
5,2.527156,횡단보도_전체면적
6,2.290877,신호등
7,1.513835,단속횟수_횡단보도위
8,1.749845,단속횟수
9,2.280357,traffic_sum


In [18]:
#초등학교_300m, 유치원_100m vif
vif5 = pd.DataFrame()
vif5['VIF Factor'] = [variance_inflation_factor(x_reg_5.values, i) for i in range(x_reg_5.shape[1])]
vif5['features'] = x_reg_5.columns
vif5

,VIF Factor,features
0,3.895926,const
1,1.269926,단속카메라
2,1.647594,도로안내표지
3,1.218469,과속방지턱
4,1.794854,스쿨존
5,2.527707,횡단보도_전체면적
6,2.286738,신호등
7,1.513177,단속횟수_횡단보도위
8,1.748973,단속횟수
9,2.285796,traffic_sum


In [19]:
#초등학교_300m, 유치원_300m vif
vif6 = pd.DataFrame()
vif6['VIF Factor'] = [variance_inflation_factor(x_reg_6.values, i) for i in range(x_reg_6.shape[1])]
vif6['features'] = x_reg_6.columns
vif6

,VIF Factor,features
0,3.898425,const
1,1.270693,단속카메라
2,1.646497,도로안내표지
3,1.221237,과속방지턱
4,2.126327,스쿨존
5,2.533761,횡단보도_전체면적
6,2.286998,신호등
7,1.512915,단속횟수_횡단보도위
8,1.748971,단속횟수
9,2.281560,traffic_sum


In [20]:
#초등학교_300m, 유치원_500m vif
vif7 = pd.DataFrame()
vif7['VIF Factor'] = [variance_inflation_factor(x_reg_7.values, i) for i in range(x_reg_7.shape[1])]
vif7['features'] = x_reg_7.columns
vif7

,VIF Factor,features
0,3.979494,const
1,1.269077,단속카메라
2,1.646740,도로안내표지
3,1.217805,과속방지턱
4,1.996898,스쿨존
5,2.530162,횡단보도_전체면적
6,2.287159,신호등
7,1.513022,단속횟수_횡단보도위
8,1.747899,단속횟수
9,2.283873,traffic_sum


In [21]:
#초등학교_300m, 유치원_가중치 vif
vif8 = pd.DataFrame()
vif8['VIF Factor'] = [variance_inflation_factor(x_reg_8.values, i) for i in range(x_reg_8.shape[1])]
vif8['features'] = x_reg_8.columns
vif8

,VIF Factor,features
0,3.895926,const
1,1.269926,단속카메라
2,1.647594,도로안내표지
3,1.218469,과속방지턱
4,1.794854,스쿨존
5,2.527707,횡단보도_전체면적
6,2.286738,신호등
7,1.513177,단속횟수_횡단보도위
8,1.748973,단속횟수
9,2.285796,traffic_sum


In [22]:
#초등학교_500m, 유치원_100m vif
vif9 = pd.DataFrame()
vif9['VIF Factor'] = [variance_inflation_factor(x_reg_9.values, i) for i in range(x_reg_9.shape[1])]
vif9['features'] = x_reg_9.columns
vif9

,VIF Factor,features
0,3.929259,const
1,1.273972,단속카메라
2,1.646535,도로안내표지
3,1.216280,과속방지턱
4,1.748361,스쿨존
5,2.527654,횡단보도_전체면적
6,2.286708,신호등
7,1.512951,단속횟수_횡단보도위
8,1.749960,단속횟수
9,2.282622,traffic_sum


In [23]:
#초등학교_500m, 유치원_300m vif
vif10 = pd.DataFrame()
vif10['VIF Factor'] = [variance_inflation_factor(x_reg_10.values, i) for i in range(x_reg_10.shape[1])]
vif10['features'] = x_reg_10.columns
vif10

,VIF Factor,features
0,3.923423,const
1,1.273509,단속카메라
2,1.645609,도로안내표지
3,1.221104,과속방지턱
4,2.055148,스쿨존
5,2.533186,횡단보도_전체면적
6,2.287047,신호등
7,1.512528,단속횟수_횡단보도위
8,1.750694,단속횟수
9,2.279354,traffic_sum


In [24]:
#초등학교_500m, 유치원_500m vif
vif11 = pd.DataFrame()
vif11['VIF Factor'] = [variance_inflation_factor(x_reg_11.values, i) for i in range(x_reg_11.shape[1])]
vif11['features'] = x_reg_11.columns
vif11

,VIF Factor,features
0,3.974989,const
1,1.272303,단속카메라
2,1.645907,도로안내표지
3,1.215966,과속방지턱
4,1.837549,스쿨존
5,2.528489,횡단보도_전체면적
6,2.287362,신호등
7,1.512502,단속횟수_횡단보도위
8,1.749555,단속횟수
9,2.281420,traffic_sum


In [25]:
#초등학교_500m, 유치원_가중치 vif
vif12 = pd.DataFrame()
vif12['VIF Factor'] = [variance_inflation_factor(x_reg_12.values, i) for i in range(x_reg_12.shape[1])]
vif12['features'] = x_reg_12.columns
vif12

,VIF Factor,features
0,3.929259,const
1,1.273972,단속카메라
2,1.646535,도로안내표지
3,1.216280,과속방지턱
4,1.748361,스쿨존
5,2.527654,횡단보도_전체면적
6,2.286708,신호등
7,1.512951,단속횟수_횡단보도위
8,1.749960,단속횟수
9,2.282622,traffic_sum


In [26]:
#초등학교_가중치, 유치원_100m vif
vif13 = pd.DataFrame()
vif13['VIF Factor'] = [variance_inflation_factor(x_reg_13.values, i) for i in range(x_reg_13.shape[1])]
vif13['features'] = x_reg_13.columns
vif13

,VIF Factor,features
0,3.903478,const
1,1.272078,단속카메라
2,1.646694,도로안내표지
3,1.216431,과속방지턱
4,1.767888,스쿨존
5,2.525225,횡단보도_전체면적
6,2.287235,신호등
7,1.512946,단속횟수_횡단보도위
8,1.748907,단속횟수
9,2.283064,traffic_sum


In [27]:
#초등학교_가중치, 유치원_300m vif
vif14 = pd.DataFrame()
vif14['VIF Factor'] = [variance_inflation_factor(x_reg_14.values, i) for i in range(x_reg_14.shape[1])]
vif14['features'] = x_reg_14.columns
vif14

,VIF Factor,features
0,3.899456,const
1,1.272389,단속카메라
2,1.645655,도로안내표지
3,1.220219,과속방지턱
4,2.084824,스쿨존
5,2.532311,횡단보도_전체면적
6,2.287880,신호등
7,1.512475,단속횟수_횡단보도위
8,1.748885,단속횟수
9,2.278867,traffic_sum


In [28]:
#초등학교_가중치, 유치원_500m vif
vif15 = pd.DataFrame()
vif15['VIF Factor'] = [variance_inflation_factor(x_reg_15.values, i) for i in range(x_reg_15.shape[1])]
vif15['features'] = x_reg_15.columns
vif15

,VIF Factor,features
0,3.976600,const
1,1.271258,단속카메라
2,1.645927,도로안내표지
3,1.215654,과속방지턱
4,1.922709,스쿨존
5,2.527725,횡단보도_전체면적
6,2.288078,신호등
7,1.512483,단속횟수_횡단보도위
8,1.747846,단속횟수
9,2.281447,traffic_sum


In [29]:
#초등학교_가중치, 유치원_가중치 vif
vif16 = pd.DataFrame()
vif16['VIF Factor'] = [variance_inflation_factor(x_reg_16.values, i) for i in range(x_reg_16.shape[1])]
vif16['features'] = x_reg_16.columns
vif16

,VIF Factor,features
0,3.903478,const
1,1.272078,단속카메라
2,1.646694,도로안내표지
3,1.216431,과속방지턱
4,1.767888,스쿨존
5,2.525225,횡단보도_전체면적
6,2.287235,신호등
7,1.512946,단속횟수_횡단보도위
8,1.748907,단속횟수
9,2.283064,traffic_sum


### VIF가 가장 높은 변수 제거

In [30]:
#초등학교_100m, 유치원_100m, time_sum제거
x_reg_1 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_100m', '유치원_100m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_1 = y[['사고횟수']]

#초등학교_100m, 유치원_300m, time_sum제거
x_reg_2 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_100m', '유치원_300m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_2 = y[['사고횟수']]

#초등학교_100m, 유치원_500m, time_sum제거
x_reg_3 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_100m', '유치원_500m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_3 = y[['사고횟수']]

#초등학교_100m, 유치원_가중치, time_sum제거
x_reg_4 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_100m', '유치원_가중치', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_4 = y[['사고횟수']]

#초등학교_300m, 유치원_100m, time_sum제거
x_reg_5 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_300m', '유치원_100m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_5 = y[['사고횟수']]

#초등학교_300m, 유치원_300m, time_sum제거
x_reg_6 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_300m', '유치원_300m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_6 = y[['사고횟수']]

#초등학교_300m, 유치원_500m, time_sum제거
x_reg_7 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_300m', '유치원_500m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_7 = y[['사고횟수']]

#초등학교_300m, 유치원_가중치, time_sum제거
x_reg_8 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_300m', '유치원_가중치', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_8 = y[['사고횟수']]

#초등학교_500m, 유치원_100m, time_sum제거
x_reg_9 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_500m', '유치원_100m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_9 = y[['사고횟수']]

#초등학교_500m, 유치원_300m, time_sum제거
x_reg_10 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_500m', '유치원_300m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_10 = y[['사고횟수']]

#초등학교_500m, 유치원_500m, time_sum제거
x_reg_11 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_500m', '유치원_500m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_11 = y[['사고횟수']]

#초등학교_500m, 유치원_가중치, time_sum제거
x_reg_12 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_500m', '유치원_가중치', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_12 = y[['사고횟수']]

#초등학교_가중치, 유치원_100m, time_sum제거
x_reg_13 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_가중치', '유치원_100m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_13 = y[['사고횟수']]

#초등학교_가중치, 유치원_300m, time_sum제거
x_reg_14 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_가중치', '유치원_300m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_14 = y[['사고횟수']]

#초등학교_가중치, 유치원_500m, time_sum제거
x_reg_15 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_가중치', '유치원_500m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_15 = y[['사고횟수']]

#초등학교_가중치, 유치원_가중치, time_sum제거
x_reg_16 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_가중치', '유치원_가중치', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_all', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_16 = y[['사고횟수']]

### 2차 다중공선성 확인

In [31]:
#초등학교_100m, 유치원_100m vif
vif1 = pd.DataFrame()
vif1['VIF Factor'] = [variance_inflation_factor(x_reg_1.values, i) for i in range(x_reg_1.shape[1])]
vif1['features'] = x_reg_1.columns
vif1

,VIF Factor,features
0,3.894666,const
1,1.269409,단속카메라
2,1.639462,도로안내표지
3,1.206121,과속방지턱
4,1.546202,스쿨존
5,2.439817,횡단보도_전체면적
6,2.186641,신호등
7,1.513145,단속횟수_횡단보도위
8,1.748786,단속횟수
9,2.057799,traffic_sum


In [32]:
#초등학교_100m, 유치원_300m vif
vif2 = pd.DataFrame()
vif2['VIF Factor'] = [variance_inflation_factor(x_reg_2.values, i) for i in range(x_reg_2.shape[1])]
vif2['features'] = x_reg_2.columns
vif2

,VIF Factor,features
0,3.898500,const
1,1.270711,단속카메라
2,1.639016,도로안내표지
3,1.210282,과속방지턱
4,2.009194,스쿨존
5,2.448052,횡단보도_전체면적
6,2.187232,신호등
7,1.512400,단속횟수_횡단보도위
8,1.748951,단속횟수
9,2.058654,traffic_sum


In [33]:
#초등학교_100m, 유치원_500m vif
vif3 = pd.DataFrame()
vif3['VIF Factor'] = [variance_inflation_factor(x_reg_3.values, i) for i in range(x_reg_3.shape[1])]
vif3['features'] = x_reg_3.columns
vif3

,VIF Factor,features
0,3.975843,const
1,1.269015,단속카메라
2,1.639297,도로안내표지
3,1.205918,과속방지턱
4,1.827235,스쿨존
5,2.438813,횡단보도_전체면적
6,2.188339,신호등
7,1.512355,단속횟수_횡단보도위
8,1.748493,단속횟수
9,2.059953,traffic_sum


In [34]:
#초등학교_100m, 유치원_가중치 vif
vif4 = pd.DataFrame()
vif4['VIF Factor'] = [variance_inflation_factor(x_reg_4.values, i) for i in range(x_reg_4.shape[1])]
vif4['features'] = x_reg_4.columns
vif4

,VIF Factor,features
0,3.894666,const
1,1.269409,단속카메라
2,1.639462,도로안내표지
3,1.206121,과속방지턱
4,1.546202,스쿨존
5,2.439817,횡단보도_전체면적
6,2.186641,신호등
7,1.513145,단속횟수_횡단보도위
8,1.748786,단속횟수
9,2.057799,traffic_sum


In [35]:
#초등학교_300m, 유치원_100m vif
vif5 = pd.DataFrame()
vif5['VIF Factor'] = [variance_inflation_factor(x_reg_5.values, i) for i in range(x_reg_5.shape[1])]
vif5['features'] = x_reg_5.columns
vif5

,VIF Factor,features
0,3.894553,const
1,1.269911,단속카메라
2,1.640233,도로안내표지
3,1.208492,과속방지턱
4,1.791837,스쿨존
5,2.439137,횡단보도_전체면적
6,2.182700,신호등
7,1.512521,단속횟수_횡단보도위
8,1.747889,단속횟수
9,2.060750,traffic_sum


In [36]:
#초등학교_300m, 유치원_300m vif
vif6 = pd.DataFrame()
vif6['VIF Factor'] = [variance_inflation_factor(x_reg_6.values, i) for i in range(x_reg_6.shape[1])]
vif6['features'] = x_reg_6.columns
vif6

,VIF Factor,features
0,3.897489,const
1,1.270667,단속카메라
2,1.639455,도로안내표지
3,1.211274,과속방지턱
4,2.123444,스쿨존
5,2.445541,횡단보도_전체면적
6,2.182948,신호등
7,1.512323,단속횟수_횡단보도위
8,1.747963,단속횟수
9,2.059941,traffic_sum


In [37]:
#초등학교_300m, 유치원_500m vif
vif7 = pd.DataFrame()
vif7['VIF Factor'] = [variance_inflation_factor(x_reg_7.values, i) for i in range(x_reg_7.shape[1])]
vif7['features'] = x_reg_7.columns
vif7

,VIF Factor,features
0,3.977851,const
1,1.269018,단속카메라
2,1.639884,도로안내표지
3,1.208390,과속방지턱
4,1.996652,스쿨존
5,2.439351,횡단보도_전체면적
6,2.184138,신호등
7,1.512409,단속횟수_횡단보도위
8,1.746986,단속횟수
9,2.061077,traffic_sum


In [38]:
#초등학교_300m, 유치원_가중치 vif
vif8 = pd.DataFrame()
vif8['VIF Factor'] = [variance_inflation_factor(x_reg_8.values, i) for i in range(x_reg_8.shape[1])]
vif8['features'] = x_reg_8.columns
vif8

,VIF Factor,features
0,3.894553,const
1,1.269911,단속카메라
2,1.640233,도로안내표지
3,1.208492,과속방지턱
4,1.791837,스쿨존
5,2.439137,횡단보도_전체면적
6,2.182700,신호등
7,1.512521,단속횟수_횡단보도위
8,1.747889,단속횟수
9,2.060750,traffic_sum


In [39]:
#초등학교_500m, 유치원_100m vif
vif9 = pd.DataFrame()
vif9['VIF Factor'] = [variance_inflation_factor(x_reg_9.values, i) for i in range(x_reg_9.shape[1])]
vif9['features'] = x_reg_9.columns
vif9

,VIF Factor,features
0,3.927218,const
1,1.273937,단속카메라
2,1.639375,도로안내표지
3,1.205874,과속방지턱
4,1.745344,스쿨존
5,2.438966,횡단보도_전체면적
6,2.182605,신호등
7,1.512235,단속횟수_횡단보도위
8,1.748807,단속횟수
9,2.058441,traffic_sum


In [40]:
#초등학교_500m, 유치원_300m vif
vif10 = pd.DataFrame()
vif10['VIF Factor'] = [variance_inflation_factor(x_reg_10.values, i) for i in range(x_reg_10.shape[1])]
vif10['features'] = x_reg_10.columns
vif10

,VIF Factor,features
0,3.921839,const
1,1.273456,단속카메라
2,1.638734,도로안내표지
3,1.210595,과속방지턱
4,2.052026,스쿨존
5,2.445021,횡단보도_전체면적
6,2.182858,신호등
7,1.511874,단속횟수_횡단보도위
8,1.749580,단속횟수
9,2.058273,traffic_sum


In [41]:
#초등학교_500m, 유치원_500m vif
vif11 = pd.DataFrame()
vif11['VIF Factor'] = [variance_inflation_factor(x_reg_11.values, i) for i in range(x_reg_11.shape[1])]
vif11['features'] = x_reg_11.columns
vif11

,VIF Factor,features
0,3.973153,const
1,1.272227,단속카메라
2,1.639118,도로안내표지
3,1.206243,과속방지턱
4,1.837111,스쿨존
5,2.437872,횡단보도_전체면적
6,2.184081,신호등
7,1.511858,단속횟수_횡단보도위
8,1.748589,단속횟수
9,2.059402,traffic_sum


In [42]:
#초등학교_500m, 유치원_가중치 vif
vif12 = pd.DataFrame()
vif12['VIF Factor'] = [variance_inflation_factor(x_reg_12.values, i) for i in range(x_reg_12.shape[1])]
vif12['features'] = x_reg_12.columns
vif12

,VIF Factor,features
0,3.927218,const
1,1.273937,단속카메라
2,1.639375,도로안내표지
3,1.205874,과속방지턱
4,1.745344,스쿨존
5,2.438966,횡단보도_전체면적
6,2.182605,신호등
7,1.512235,단속횟수_횡단보도위
8,1.748807,단속횟수
9,2.058441,traffic_sum


In [43]:
#초등학교_가중치, 유치원_100m vif
vif13 = pd.DataFrame()
vif13['VIF Factor'] = [variance_inflation_factor(x_reg_13.values, i) for i in range(x_reg_13.shape[1])]
vif13['features'] = x_reg_13.columns
vif13

,VIF Factor,features
0,3.901847,const
1,1.272054,단속카메라
2,1.639471,도로안내표지
3,1.206172,과속방지턱
4,1.764700,스쿨존
5,2.437293,횡단보도_전체면적
6,2.182446,신호등
7,1.512233,단속횟수_횡단보도위
8,1.747831,단속횟수
9,2.058867,traffic_sum


In [44]:
#초등학교_가중치, 유치원_300m vif
vif14 = pd.DataFrame()
vif14['VIF Factor'] = [variance_inflation_factor(x_reg_14.values, i) for i in range(x_reg_14.shape[1])]
vif14['features'] = x_reg_14.columns
vif14

,VIF Factor,features
0,3.898357,const
1,1.272354,단속카메라
2,1.638756,도로안내표지
3,1.209965,과속방지턱
4,2.081532,스쿨존
5,2.444826,횡단보도_전체면적
6,2.183133,신호등
7,1.511835,단속횟수_횡단보도위
8,1.747888,단속횟수
9,2.058318,traffic_sum


In [45]:
#초등학교_가중치, 유치원_500m vif
vif15 = pd.DataFrame()
vif15['VIF Factor'] = [variance_inflation_factor(x_reg_15.values, i) for i in range(x_reg_15.shape[1])]
vif15['features'] = x_reg_15.columns
vif15

,VIF Factor,features
0,3.974864,const
1,1.271200,단속카메라
2,1.639160,도로안내표지
3,1.206055,과속방지턱
4,1.922227,스쿨존
5,2.437464,횡단보도_전체면적
6,2.184801,신호등
7,1.511844,단속횟수_횡단보도위
8,1.746941,단속횟수
9,2.059490,traffic_sum


In [46]:
#초등학교_가중치, 유치원_가중치 vif
vif16 = pd.DataFrame()
vif16['VIF Factor'] = [variance_inflation_factor(x_reg_16.values, i) for i in range(x_reg_16.shape[1])]
vif16['features'] = x_reg_16.columns
vif16

,VIF Factor,features
0,3.901847,const
1,1.272054,단속카메라
2,1.639471,도로안내표지
3,1.206172,과속방지턱
4,1.764700,스쿨존
5,2.437293,횡단보도_전체면적
6,2.182446,신호등
7,1.512233,단속횟수_횡단보도위
8,1.747831,단속횟수
9,2.058867,traffic_sum


### VIF가 가장 높은 변수 제거

In [47]:
#초등학교_100m, 유치원_100m, time_sum제거, pop_all제거
x_reg_1 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_100m', '유치원_100m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_1 = y[['사고횟수']]

#초등학교_100m, 유치원_300m, time_sum제거, pop_all제거
x_reg_2 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_100m', '유치원_300m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_2 = y[['사고횟수']]

#초등학교_100m, 유치원_500m, time_sum제거, pop_all제거
x_reg_3 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_100m', '유치원_500m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_3 = y[['사고횟수']]

#초등학교_100m, 유치원_가중치, time_sum제거, pop_all제거
x_reg_4 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_100m', '유치원_가중치', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_4 = y[['사고횟수']]

#초등학교_300m, 유치원_100m, time_sum제거, pop_all제거
x_reg_5 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_300m', '유치원_100m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_5 = y[['사고횟수']]

#초등학교_300m, 유치원_300m, time_sum제거, pop_all제거
x_reg_6 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_300m', '유치원_300m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_6 = y[['사고횟수']]

#초등학교_300m, 유치원_500m, time_sum제거, pop_all제거
x_reg_7 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_300m', '유치원_500m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_7 = y[['사고횟수']]

#초등학교_300m, 유치원_가중치, time_sum제거, pop_all제거
x_reg_8 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_300m', '유치원_가중치', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_8 = y[['사고횟수']]

#초등학교_500m, 유치원_100m, time_sum제거, pop_all제거
x_reg_9 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_500m', '유치원_100m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_9 = y[['사고횟수']]

#초등학교_500m, 유치원_300m, time_sum제거, pop_all제거
x_reg_10 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_500m', '유치원_300m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_10 = y[['사고횟수']]

#초등학교_500m, 유치원_500m, time_sum제거, pop_all제거
x_reg_11 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_500m', '유치원_500m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_11 = y[['사고횟수']]

#초등학교_500m, 유치원_가중치, time_sum제거, pop_all제거
x_reg_12 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_500m', '유치원_가중치', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_12 = y[['사고횟수']]

#초등학교_가중치, 유치원_100m, time_sum제거, pop_all제거
x_reg_13 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_가중치', '유치원_100m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_13 = y[['사고횟수']]

#초등학교_가중치, 유치원_300m, time_sum제거, pop_all제거
x_reg_14 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_가중치', '유치원_300m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_14 = y[['사고횟수']]

#초등학교_가중치, 유치원_500m, time_sum제거, pop_all제거
x_reg_15 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_가중치', '유치원_500m', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_15 = y[['사고횟수']]

#초등학교_가중치, 유치원_가중치, time_sum제거, pop_all제거
x_reg_16 = X[['const', '단속카메라', '도로안내표지', '과속방지턱', '스쿨존', '횡단보도_전체면적', '신호등', '단속횟수_횡단보도위', '단속횟수',
                     'traffic_sum', 'freq_sum', '초등학교_가중치', '유치원_가중치', '유동인구', 'cctv', '버스정류장',
                     '체육시설', '학원', 'car_cnt', 'pop_child', 'region_type_HS', 'region_type_OF', 'float_type_H', 'float_type_M'
                     ]]

y_reg_16 = y[['사고횟수']]

### 3차 다중공선성 확인

In [48]:
#초등학교_100m, 유치원_100m vif >> ok
vif1 = pd.DataFrame()
vif1['VIF Factor'] = [variance_inflation_factor(x_reg_1.values, i) for i in range(x_reg_1.shape[1])]
vif1['features'] = x_reg_1.columns
vif1

,VIF Factor,features
0,3.889782,const
1,1.268960,단속카메라
2,1.635754,도로안내표지
3,1.188847,과속방지턱
4,1.512753,스쿨존
5,2.436559,횡단보도_전체면적
6,2.186500,신호등
7,1.513096,단속횟수_횡단보도위
8,1.740916,단속횟수
9,2.049603,traffic_sum


In [49]:
#초등학교_100m, 유치원_300m vif >> ok
vif2 = pd.DataFrame()
vif2['VIF Factor'] = [variance_inflation_factor(x_reg_2.values, i) for i in range(x_reg_2.shape[1])]
vif2['features'] = x_reg_2.columns
vif2

,VIF Factor,features
0,3.892371,const
1,1.270229,단속카메라
2,1.635069,도로안내표지
3,1.192809,과속방지턱
4,1.977781,스쿨존
5,2.444791,횡단보도_전체면적
6,2.187082,신호등
7,1.512317,단속횟수_횡단보도위
8,1.741120,단속횟수
9,2.049823,traffic_sum


In [50]:
#초등학교_100m, 유치원_500m vif >> ok
vif3 = pd.DataFrame()
vif3['VIF Factor'] = [variance_inflation_factor(x_reg_3.values, i) for i in range(x_reg_3.shape[1])]
vif3['features'] = x_reg_3.columns
vif3

,VIF Factor,features
0,3.968531,const
1,1.268464,단속카메라
2,1.635417,도로안내표지
3,1.189095,과속방지턱
4,1.789789,스쿨존
5,2.435953,횡단보도_전체면적
6,2.188190,신호등
7,1.512263,단속횟수_횡단보도위
8,1.740814,단속횟수
9,2.051084,traffic_sum


In [51]:
#초등학교_100m, 유치원_가중치 vif >> ok
vif4 = pd.DataFrame()
vif4['VIF Factor'] = [variance_inflation_factor(x_reg_4.values, i) for i in range(x_reg_4.shape[1])]
vif4['features'] = x_reg_4.columns
vif4

,VIF Factor,features
0,3.889782,const
1,1.268960,단속카메라
2,1.635754,도로안내표지
3,1.188847,과속방지턱
4,1.512753,스쿨존
5,2.436559,횡단보도_전체면적
6,2.186500,신호등
7,1.513096,단속횟수_횡단보도위
8,1.740916,단속횟수
9,2.049603,traffic_sum


In [52]:
#초등학교_300m, 유치원_100m vif >> ok
vif5 = pd.DataFrame()
vif5['VIF Factor'] = [variance_inflation_factor(x_reg_5.values, i) for i in range(x_reg_5.shape[1])]
vif5['features'] = x_reg_5.columns
vif5

,VIF Factor,features
0,3.889687,const
1,1.269495,단속카메라
2,1.636478,도로안내표지
3,1.191040,과속방지턱
4,1.761889,스쿨존
5,2.435881,횡단보도_전체면적
6,2.182541,신호등
7,1.512469,단속횟수_횡단보도위
8,1.739963,단속횟수
9,2.052404,traffic_sum


In [53]:
#초등학교_300m, 유치원_300m vif >> ok
vif6 = pd.DataFrame()
vif6['VIF Factor'] = [variance_inflation_factor(x_reg_6.values, i) for i in range(x_reg_6.shape[1])]
vif6['features'] = x_reg_6.columns
vif6

,VIF Factor,features
0,3.891433,const
1,1.270199,단속카메라
2,1.635442,도로안내표지
3,1.193616,과속방지턱
4,2.094206,스쿨존
5,2.442207,횡단보도_전체면적
6,2.182808,신호등
7,1.512248,단속횟수_횡단보도위
8,1.740200,단속횟수
9,2.050948,traffic_sum


In [54]:
#초등학교_300m, 유치원_500m vif >> ok
vif7 = pd.DataFrame()
vif7['VIF Factor'] = [variance_inflation_factor(x_reg_7.values, i) for i in range(x_reg_7.shape[1])]
vif7['features'] = x_reg_7.columns
vif7

,VIF Factor,features
0,3.970772,const
1,1.268489,단속카메라
2,1.635891,도로안내표지
3,1.191220,과속방지턱
4,1.962993,스쿨존
5,2.436319,횡단보도_전체면적
6,2.184011,신호등
7,1.512332,단속횟수_횡단보도위
8,1.739416,단속횟수
9,2.051965,traffic_sum


In [55]:
#초등학교_300m, 유치원_가중치 vif >> ok
vif8 = pd.DataFrame()
vif8['VIF Factor'] = [variance_inflation_factor(x_reg_8.values, i) for i in range(x_reg_8.shape[1])]
vif8['features'] = x_reg_8.columns
vif8

,VIF Factor,features
0,3.889687,const
1,1.269495,단속카메라
2,1.636478,도로안내표지
3,1.191040,과속방지턱
4,1.761889,스쿨존
5,2.435881,횡단보도_전체면적
6,2.182541,신호등
7,1.512469,단속횟수_횡단보도위
8,1.739963,단속횟수
9,2.052404,traffic_sum


In [56]:
#초등학교_500m, 유치원_100m vif
vif9 = pd.DataFrame()
vif9['VIF Factor'] = [variance_inflation_factor(x_reg_9.values, i) for i in range(x_reg_9.shape[1])]
vif9['features'] = x_reg_9.columns
vif9

,VIF Factor,features
0,3.920579,const
1,1.273671,단속카메라
2,1.635700,도로안내표지
3,1.188741,과속방지턱
4,1.722165,스쿨존
5,2.435467,횡단보도_전체면적
6,2.182464,신호등
7,1.512170,단속횟수_횡단보도위
8,1.741250,단속횟수
9,2.049978,traffic_sum


In [57]:
#초등학교_500m, 유치원_300m vif
vif10 = pd.DataFrame()
vif10['VIF Factor'] = [variance_inflation_factor(x_reg_10.values, i) for i in range(x_reg_10.shape[1])]
vif10['features'] = x_reg_10.columns
vif10

,VIF Factor,features
0,3.914146,const
1,1.273110,단속카메라
2,1.634804,도로안내표지
3,1.193566,과속방지턱
4,2.024928,스쿨존
5,2.441653,횡단보도_전체면적
6,2.182726,신호등
7,1.511784,단속횟수_횡단보도위
8,1.742237,단속횟수
9,2.049284,traffic_sum


In [58]:
#초등학교_500m, 유치원_500m vif
vif11 = pd.DataFrame()
vif11['VIF Factor'] = [variance_inflation_factor(x_reg_11.values, i) for i in range(x_reg_11.shape[1])]
vif11['features'] = x_reg_11.columns
vif11

,VIF Factor,features
0,3.965459,const
1,1.271842,단속카메라
2,1.635168,도로안내표지
3,1.189785,과속방지턱
4,1.803166,스쿨존
5,2.434803,횡단보도_전체면적
6,2.183938,신호등
7,1.511765,단속횟수_횡단보도위
8,1.741456,단속횟수
9,2.050516,traffic_sum


In [59]:
#초등학교_500m, 유치원_가중치 vif
vif12 = pd.DataFrame()
vif12['VIF Factor'] = [variance_inflation_factor(x_reg_12.values, i) for i in range(x_reg_12.shape[1])]
vif12['features'] = x_reg_12.columns
vif12

,VIF Factor,features
0,3.920579,const
1,1.273671,단속카메라
2,1.635700,도로안내표지
3,1.188741,과속방지턱
4,1.722165,스쿨존
5,2.435467,횡단보도_전체면적
6,2.182464,신호등
7,1.512170,단속횟수_횡단보도위
8,1.741250,단속횟수
9,2.049978,traffic_sum


In [60]:
#초등학교_가중치, 유치원_100m vif
vif13 = pd.DataFrame()
vif13['VIF Factor'] = [variance_inflation_factor(x_reg_13.values, i) for i in range(x_reg_13.shape[1])]
vif13['features'] = x_reg_13.columns
vif13

,VIF Factor,features
0,3.896576,const
1,1.271691,단속카메라
2,1.635744,도로안내표지
3,1.188827,과속방지턱
4,1.737141,스쿨존
5,2.434051,횡단보도_전체면적
6,2.182269,신호등
7,1.512172,단속횟수_횡단보도위
8,1.739924,단속횟수
9,2.050497,traffic_sum


In [61]:
#초등학교_가중치, 유치원_300m vif
vif14 = pd.DataFrame()
vif14['VIF Factor'] = [variance_inflation_factor(x_reg_14.values, i) for i in range(x_reg_14.shape[1])]
vif14['features'] = x_reg_14.columns
vif14

,VIF Factor,features
0,3.898357,const
1,1.272354,단속카메라
2,1.638756,도로안내표지
3,1.209965,과속방지턱
4,2.081532,스쿨존
5,2.444826,횡단보도_전체면적
6,2.183133,신호등
7,1.511835,단속횟수_횡단보도위
8,1.747888,단속횟수
9,2.058318,traffic_sum


In [62]:
#초등학교_가중치, 유치원_500m vif
vif15 = pd.DataFrame()
vif15['VIF Factor'] = [variance_inflation_factor(x_reg_15.values, i) for i in range(x_reg_15.shape[1])]
vif15['features'] = x_reg_15.columns
vif15

,VIF Factor,features
0,3.967848,const
1,1.270748,단속카메라
2,1.635208,도로안내표지
3,1.189095,과속방지턱
4,1.889989,스쿨존
5,2.434491,횡단보도_전체면적
6,2.184637,신호등
7,1.511755,단속횟수_횡단보도위
8,1.739376,단속횟수
9,2.050513,traffic_sum


In [63]:
#초등학교_가중치, 유치원_가중치 vif
vif16 = pd.DataFrame()
vif16['VIF Factor'] = [variance_inflation_factor(x_reg_16.values, i) for i in range(x_reg_16.shape[1])]
vif16['features'] = x_reg_16.columns
vif16

,VIF Factor,features
0,3.896576,const
1,1.271691,단속카메라
2,1.635744,도로안내표지
3,1.188827,과속방지턱
4,1.737141,스쿨존
5,2.434051,횡단보도_전체면적
6,2.182269,신호등
7,1.512172,단속횟수_횡단보도위
8,1.739924,단속횟수
9,2.050497,traffic_sum


모든 모형에서 pop_all, time_sum변수 제거 후 독립변수 간 다중공선성이 없음을 확인할 수 있다.

## 공간가중행렬 정의

In [14]:
#Queen, DistanceBand 방식으로 정의

W_Q = Queen.from_dataframe(osan)
W_D = DistanceBand.from_dataframe(osan, threshold=0.005, binary=False) #500m

In [15]:
#행 기준으로 공간가중행렬 정규화

W_Q.transform = 'r'
W_D.transfomr = 'r'

## 모형에 사용될 데이터 변환 및 정의

In [66]:
#데이터 형태를 array로 변환

x_1 = np.array(x_reg_1)
y_1 = np.array(y_reg_1)

x_2 = np.array(x_reg_2)
y_2 = np.array(y_reg_2)

x_3 = np.array(x_reg_3)
y_3 = np.array(y_reg_3)

x_4 = np.array(x_reg_4)
y_4 = np.array(y_reg_4)

x_5 = np.array(x_reg_5)
y_5 = np.array(y_reg_5)

x_6 = np.array(x_reg_6)
y_6 = np.array(y_reg_6)

x_7 = np.array(x_reg_7)
y_7 = np.array(y_reg_7)

x_8 = np.array(x_reg_8)
y_8 = np.array(y_reg_8)

x_9 = np.array(x_reg_9)
y_9 = np.array(y_reg_9)

x_10 = np.array(x_reg_10)
y_10 = np.array(y_reg_10)

x_11 = np.array(x_reg_11)
y_11 = np.array(y_reg_11)

x_12 = np.array(x_reg_12)
y_12 = np.array(y_reg_12)

x_13 = np.array(x_reg_13)
y_13 = np.array(y_reg_13)

x_14 = np.array(x_reg_14)
y_14 = np.array(y_reg_14)

x_15 = np.array(x_reg_15)
y_15 = np.array(y_reg_15)

x_16 = np.array(x_reg_16)
y_16 = np.array(y_reg_16)

In [67]:
#모델에서 사용될 변수이름 정의

name_x_1 = list(x_reg_1.columns)
name_y_1 = list(y_reg_1.columns)

name_x_2 = list(x_reg_2.columns)
name_y_2 = list(y_reg_2.columns)

name_x_3 = list(x_reg_3.columns)
name_y_3 = list(y_reg_3.columns)

name_x_4 = list(x_reg_4.columns)
name_y_4 = list(y_reg_4.columns)

name_x_5 = list(x_reg_5.columns)
name_y_5 = list(y_reg_5.columns)

name_x_6 = list(x_reg_6.columns)
name_y_6 = list(y_reg_6.columns)

name_x_7 = list(x_reg_7.columns)
name_y_7 = list(y_reg_7.columns)

name_x_8 = list(x_reg_8.columns)
name_y_8 = list(y_reg_8.columns)

name_x_9 = list(x_reg_9.columns)
name_y_9 = list(y_reg_9.columns)

name_x_10 = list(x_reg_10.columns)
name_y_10 = list(y_reg_10.columns)

name_x_11 = list(x_reg_11.columns)
name_y_11 = list(y_reg_11.columns)

name_x_12 = list(x_reg_12.columns)
name_y_12 = list(y_reg_12.columns)

name_x_13 = list(x_reg_13.columns)
name_y_13 = list(y_reg_13.columns)

name_x_14 = list(x_reg_14.columns)
name_y_14 = list(y_reg_14.columns)

name_x_15 = list(x_reg_15.columns)
name_y_15 = list(y_reg_15.columns)

name_x_16 = list(x_reg_16.columns)
name_y_16 = list(y_reg_16.columns)

## OLS모형 적합 및 공간자기상관 확인

### OLS모형 적합

In [68]:
#Queen방식의 공간가중행렬을 사용한 OLS 적합

ols_Q_1 = OLS(y=y_1, x=x_1, w=W_Q, name_x=name_x_1, name_y=name_y_1, white_test=True, spat_diag=True, moran=True)

ols_Q_2 = OLS(y=y_2, x=x_2, w=W_Q, name_x=name_x_2, name_y=name_y_2, white_test=True, spat_diag=True, moran=True)

ols_Q_3 = OLS(y=y_3, x=x_3, w=W_Q, name_x=name_x_3, name_y=name_y_3, white_test=True, spat_diag=True, moran=True)

ols_Q_4 = OLS(y=y_4, x=x_4, w=W_Q, name_x=name_x_4, name_y=name_y_4, white_test=True, spat_diag=True, moran=True)

ols_Q_5 = OLS(y=y_5, x=x_5, w=W_Q, name_x=name_x_5, name_y=name_y_5, white_test=True, spat_diag=True, moran=True)

ols_Q_6 = OLS(y=y_6, x=x_6, w=W_Q, name_x=name_x_6, name_y=name_y_6, white_test=True, spat_diag=True, moran=True)

ols_Q_7 = OLS(y=y_7, x=x_7, w=W_Q, name_x=name_x_7, name_y=name_y_7, white_test=True, spat_diag=True, moran=True)

ols_Q_8 = OLS(y=y_8, x=x_8, w=W_Q, name_x=name_x_8, name_y=name_y_8, white_test=True, spat_diag=True, moran=True)

ols_Q_9 = OLS(y=y_9, x=x_9, w=W_Q, name_x=name_x_9, name_y=name_y_9, white_test=True, spat_diag=True, moran=True)

ols_Q_10 = OLS(y=y_10, x=x_10, w=W_Q, name_x=name_x_10, name_y=name_y_10, white_test=True, spat_diag=True, moran=True)

ols_Q_11 = OLS(y=y_11, x=x_11, w=W_Q, name_x=name_x_11, name_y=name_y_11, white_test=True, spat_diag=True, moran=True)

ols_Q_12 = OLS(y=y_12, x=x_12, w=W_Q, name_x=name_x_12, name_y=name_y_12, white_test=True, spat_diag=True, moran=True)

ols_Q_13 = OLS(y=y_13, x=x_13, w=W_Q, name_x=name_x_13, name_y=name_y_13, white_test=True, spat_diag=True, moran=True)

ols_Q_14 = OLS(y=y_14, x=x_14, w=W_Q, name_x=name_x_14, name_y=name_y_14, white_test=True, spat_diag=True, moran=True)

ols_Q_15 = OLS(y=y_15, x=x_15, w=W_Q, name_x=name_x_15, name_y=name_y_15, white_test=True, spat_diag=True, moran=True)

ols_Q_16 = OLS(y=y_16, x=x_16, w=W_Q, name_x=name_x_16, name_y=name_y_16, white_test=True, spat_diag=True, moran=True)

In [69]:
#DistanceBand방식의 공간가중행렬을 사용한 OLS 적합

ols_D_1 = OLS(y=y_1, x=x_1, w=W_D, name_x=name_x_1, name_y=name_y_1, white_test=True, spat_diag=True, moran=True)

ols_D_2 = OLS(y=y_2, x=x_2, w=W_D, name_x=name_x_2, name_y=name_y_2, white_test=True, spat_diag=True, moran=True)

ols_D_3 = OLS(y=y_3, x=x_3, w=W_D, name_x=name_x_3, name_y=name_y_3, white_test=True, spat_diag=True, moran=True)

ols_D_4 = OLS(y=y_4, x=x_4, w=W_D, name_x=name_x_4, name_y=name_y_4, white_test=True, spat_diag=True, moran=True)

ols_D_5 = OLS(y=y_5, x=x_5, w=W_D, name_x=name_x_5, name_y=name_y_5, white_test=True, spat_diag=True, moran=True)

ols_D_6 = OLS(y=y_6, x=x_6, w=W_D, name_x=name_x_6, name_y=name_y_6, white_test=True, spat_diag=True, moran=True)

ols_D_7 = OLS(y=y_7, x=x_7, w=W_D, name_x=name_x_7, name_y=name_y_7, white_test=True, spat_diag=True, moran=True)

ols_D_8 = OLS(y=y_8, x=x_8, w=W_D, name_x=name_x_8, name_y=name_y_8, white_test=True, spat_diag=True, moran=True)

ols_D_9 = OLS(y=y_9, x=x_9, w=W_D, name_x=name_x_9, name_y=name_y_9, white_test=True, spat_diag=True, moran=True)

ols_D_10 = OLS(y=y_10, x=x_10, w=W_D, name_x=name_x_10, name_y=name_y_10, white_test=True, spat_diag=True, moran=True)

ols_D_11 = OLS(y=y_11, x=x_11, w=W_D, name_x=name_x_11, name_y=name_y_11, white_test=True, spat_diag=True, moran=True)

ols_D_12 = OLS(y=y_12, x=x_12, w=W_D, name_x=name_x_12, name_y=name_y_12, white_test=True, spat_diag=True, moran=True)

ols_D_13 = OLS(y=y_13, x=x_13, w=W_D, name_x=name_x_13, name_y=name_y_13, white_test=True, spat_diag=True, moran=True)

ols_D_14 = OLS(y=y_14, x=x_14, w=W_D, name_x=name_x_14, name_y=name_y_14, white_test=True, spat_diag=True, moran=True)

ols_D_15 = OLS(y=y_15, x=x_15, w=W_D, name_x=name_x_15, name_y=name_y_15, white_test=True, spat_diag=True, moran=True)

ols_D_16 = OLS(y=y_16, x=x_16, w=W_D, name_x=name_x_16, name_y=name_y_16, white_test=True, spat_diag=True, moran=True)

### OLS모형의 요약 정보 및 공간자기상관 확인

#### Queen방식의 공간가중행렬을 사용한 OLS 요약 정보

In [70]:
print(ols_Q_1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3528
Adjusted R-squared  :      0.3494
Sum squared residual:     773.500                F-statistic           :    106.2994
Sigma-square        :       0.172                Prob(F-statistic)     :           0
S.E. of regression  :       0.415                Log likelihood        :   -2423.561
Sigma-square ML     :       0.172                Akaike info criterion :    4895.123
S.E of regression ML:      0.4141                Schwarz criterion     :    5049.060

-----------------------------------------------------------------------------

In [71]:
print(ols_Q_2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3557
Adjusted R-squared  :      0.3524
Sum squared residual:     769.984                F-statistic           :    107.6756
Sigma-square        :       0.172                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2413.287
Sigma-square ML     :       0.171                Akaike info criterion :    4874.574
S.E of regression ML:      0.4132                Schwarz criterion     :    5028.511

-----------------------------------------------------------------------------

In [72]:
print(ols_Q_3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3594
Adjusted R-squared  :      0.3561
Sum squared residual:     765.523                F-statistic           :    109.4393
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.413                Log likelihood        :   -2400.186
Sigma-square ML     :       0.170                Akaike info criterion :    4848.373
S.E of regression ML:      0.4120                Schwarz criterion     :    5002.310

-----------------------------------------------------------------------------

In [73]:
print(ols_Q_4.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3528
Adjusted R-squared  :      0.3494
Sum squared residual:     773.500                F-statistic           :    106.2994
Sigma-square        :       0.172                Prob(F-statistic)     :           0
S.E. of regression  :       0.415                Log likelihood        :   -2423.561
Sigma-square ML     :       0.172                Akaike info criterion :    4895.123
S.E of regression ML:      0.4141                Schwarz criterion     :    5049.060

-----------------------------------------------------------------------------

In [74]:
print(ols_Q_5.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3567
Adjusted R-squared  :      0.3534
Sum squared residual:     768.829                F-statistic           :    108.1300
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2409.904
Sigma-square ML     :       0.170                Akaike info criterion :    4867.808
S.E of regression ML:      0.4129                Schwarz criterion     :    5021.745

-----------------------------------------------------------------------------

In [75]:
print(ols_Q_6.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3582
Adjusted R-squared  :      0.3550
Sum squared residual:     766.940                F-statistic           :    108.8771
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.413                Log likelihood        :   -2404.354
Sigma-square ML     :       0.170                Akaike info criterion :    4856.709
S.E of regression ML:      0.4124                Schwarz criterion     :    5010.646

-----------------------------------------------------------------------------

In [76]:
print(ols_Q_7.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3616
Adjusted R-squared  :      0.3583
Sum squared residual:     762.900                F-statistic           :    110.4864
Sigma-square        :       0.170                Prob(F-statistic)     :           0
S.E. of regression  :       0.412                Log likelihood        :   -2392.445
Sigma-square ML     :       0.169                Akaike info criterion :    4832.890
S.E of regression ML:      0.4113                Schwarz criterion     :    4986.828

-----------------------------------------------------------------------------

In [77]:
print(ols_Q_8.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3567
Adjusted R-squared  :      0.3534
Sum squared residual:     768.829                F-statistic           :    108.1300
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2409.904
Sigma-square ML     :       0.170                Akaike info criterion :    4867.808
S.E of regression ML:      0.4129                Schwarz criterion     :    5021.745

-----------------------------------------------------------------------------

In [78]:
print(ols_Q_9.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3562
Adjusted R-squared  :      0.3529
Sum squared residual:     769.385                F-statistic           :    107.9113
Sigma-square        :       0.172                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2411.532
Sigma-square ML     :       0.171                Akaike info criterion :    4871.064
S.E of regression ML:      0.4130                Schwarz criterion     :    5025.001

-----------------------------------------------------------------------------

In [79]:
print(ols_Q_10.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3576
Adjusted R-squared  :      0.3543
Sum squared residual:     767.712                F-statistic           :    108.5712
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2406.625
Sigma-square ML     :       0.170                Akaike info criterion :    4861.250
S.E of regression ML:      0.4126                Schwarz criterion     :    5015.187

-----------------------------------------------------------------------------

In [80]:
print(ols_Q_11.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3596
Adjusted R-squared  :      0.3563
Sum squared residual:     765.296                F-statistic           :    109.5298
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.413                Log likelihood        :   -2399.517
Sigma-square ML     :       0.170                Akaike info criterion :    4847.033
S.E of regression ML:      0.4119                Schwarz criterion     :    5000.970

-----------------------------------------------------------------------------

In [81]:
print(ols_Q_12.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3562
Adjusted R-squared  :      0.3529
Sum squared residual:     769.385                F-statistic           :    107.9113
Sigma-square        :       0.172                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2411.532
Sigma-square ML     :       0.171                Akaike info criterion :    4871.064
S.E of regression ML:      0.4130                Schwarz criterion     :    5025.001

-----------------------------------------------------------------------------

In [82]:
print(ols_Q_13.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3576
Adjusted R-squared  :      0.3543
Sum squared residual:     767.692                F-statistic           :    108.5793
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2406.565
Sigma-square ML     :       0.170                Akaike info criterion :    4861.130
S.E of regression ML:      0.4126                Schwarz criterion     :    5015.067

-----------------------------------------------------------------------------

In [29]:
print(ols_Q_14.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3585
Adjusted R-squared  :      0.3552
Sum squared residual:     766.661                F-statistic           :    108.9874
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.413                Log likelihood        :   -2403.536
Sigma-square ML     :       0.170                Akaike info criterion :    4855.072
S.E of regression ML:      0.4123                Schwarz criterion     :    5009.009

-----------------------------------------------------------------------------

In [84]:
print(ols_Q_15.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3610
Adjusted R-squared  :      0.3577
Sum squared residual:     763.626                F-statistic           :    110.1959
Sigma-square        :       0.170                Prob(F-statistic)     :           0
S.E. of regression  :       0.413                Log likelihood        :   -2394.591
Sigma-square ML     :       0.169                Akaike info criterion :    4837.181
S.E of regression ML:      0.4115                Schwarz criterion     :    4991.119

-----------------------------------------------------------------------------

In [85]:
print(ols_Q_16.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3576
Adjusted R-squared  :      0.3543
Sum squared residual:     767.692                F-statistic           :    108.5793
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2406.565
Sigma-square ML     :       0.170                Akaike info criterion :    4861.130
S.E of regression ML:      0.4126                Schwarz criterion     :    5015.067

-----------------------------------------------------------------------------

#### DistanceBand방식의 공간가중행렬을 사용한 OLS 요약 정보

In [86]:
print(ols_D_1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3528
Adjusted R-squared  :      0.3494
Sum squared residual:     773.500                F-statistic           :    106.2994
Sigma-square        :       0.172                Prob(F-statistic)     :           0
S.E. of regression  :       0.415                Log likelihood        :   -2423.561
Sigma-square ML     :       0.172                Akaike info criterion :    4895.123
S.E of regression ML:      0.4141                Schwarz criterion     :    5049.060

-----------------------------------------------------------------------------

In [87]:
print(ols_D_2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3557
Adjusted R-squared  :      0.3524
Sum squared residual:     769.984                F-statistic           :    107.6756
Sigma-square        :       0.172                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2413.287
Sigma-square ML     :       0.171                Akaike info criterion :    4874.574
S.E of regression ML:      0.4132                Schwarz criterion     :    5028.511

-----------------------------------------------------------------------------

In [88]:
print(ols_D_3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3594
Adjusted R-squared  :      0.3561
Sum squared residual:     765.523                F-statistic           :    109.4393
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.413                Log likelihood        :   -2400.186
Sigma-square ML     :       0.170                Akaike info criterion :    4848.373
S.E of regression ML:      0.4120                Schwarz criterion     :    5002.310

-----------------------------------------------------------------------------

In [89]:
print(ols_D_4.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3528
Adjusted R-squared  :      0.3494
Sum squared residual:     773.500                F-statistic           :    106.2994
Sigma-square        :       0.172                Prob(F-statistic)     :           0
S.E. of regression  :       0.415                Log likelihood        :   -2423.561
Sigma-square ML     :       0.172                Akaike info criterion :    4895.123
S.E of regression ML:      0.4141                Schwarz criterion     :    5049.060

-----------------------------------------------------------------------------

In [90]:
print(ols_D_5.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3567
Adjusted R-squared  :      0.3534
Sum squared residual:     768.829                F-statistic           :    108.1300
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2409.904
Sigma-square ML     :       0.170                Akaike info criterion :    4867.808
S.E of regression ML:      0.4129                Schwarz criterion     :    5021.745

-----------------------------------------------------------------------------

In [91]:
print(ols_D_6.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3582
Adjusted R-squared  :      0.3550
Sum squared residual:     766.940                F-statistic           :    108.8771
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.413                Log likelihood        :   -2404.354
Sigma-square ML     :       0.170                Akaike info criterion :    4856.709
S.E of regression ML:      0.4124                Schwarz criterion     :    5010.646

-----------------------------------------------------------------------------

In [92]:
print(ols_D_7.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3616
Adjusted R-squared  :      0.3583
Sum squared residual:     762.900                F-statistic           :    110.4864
Sigma-square        :       0.170                Prob(F-statistic)     :           0
S.E. of regression  :       0.412                Log likelihood        :   -2392.445
Sigma-square ML     :       0.169                Akaike info criterion :    4832.890
S.E of regression ML:      0.4113                Schwarz criterion     :    4986.828

-----------------------------------------------------------------------------

In [93]:
print(ols_D_8.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3567
Adjusted R-squared  :      0.3534
Sum squared residual:     768.829                F-statistic           :    108.1300
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2409.904
Sigma-square ML     :       0.170                Akaike info criterion :    4867.808
S.E of regression ML:      0.4129                Schwarz criterion     :    5021.745

-----------------------------------------------------------------------------

In [94]:
print(ols_D_9.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3562
Adjusted R-squared  :      0.3529
Sum squared residual:     769.385                F-statistic           :    107.9113
Sigma-square        :       0.172                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2411.532
Sigma-square ML     :       0.171                Akaike info criterion :    4871.064
S.E of regression ML:      0.4130                Schwarz criterion     :    5025.001

-----------------------------------------------------------------------------

In [95]:
print(ols_D_10.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3576
Adjusted R-squared  :      0.3543
Sum squared residual:     767.712                F-statistic           :    108.5712
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2406.625
Sigma-square ML     :       0.170                Akaike info criterion :    4861.250
S.E of regression ML:      0.4126                Schwarz criterion     :    5015.187

-----------------------------------------------------------------------------

In [96]:
print(ols_D_11.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3596
Adjusted R-squared  :      0.3563
Sum squared residual:     765.296                F-statistic           :    109.5298
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.413                Log likelihood        :   -2399.517
Sigma-square ML     :       0.170                Akaike info criterion :    4847.033
S.E of regression ML:      0.4119                Schwarz criterion     :    5000.970

-----------------------------------------------------------------------------

In [97]:
print(ols_D_12.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3562
Adjusted R-squared  :      0.3529
Sum squared residual:     769.385                F-statistic           :    107.9113
Sigma-square        :       0.172                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2411.532
Sigma-square ML     :       0.171                Akaike info criterion :    4871.064
S.E of regression ML:      0.4130                Schwarz criterion     :    5025.001

-----------------------------------------------------------------------------

In [98]:
print(ols_D_13.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3576
Adjusted R-squared  :      0.3543
Sum squared residual:     767.692                F-statistic           :    108.5793
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2406.565
Sigma-square ML     :       0.170                Akaike info criterion :    4861.130
S.E of regression ML:      0.4126                Schwarz criterion     :    5015.067

-----------------------------------------------------------------------------

In [30]:
print(ols_D_14.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3585
Adjusted R-squared  :      0.3552
Sum squared residual:     766.661                F-statistic           :    108.9874
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.413                Log likelihood        :   -2403.536
Sigma-square ML     :       0.170                Akaike info criterion :    4855.072
S.E of regression ML:      0.4123                Schwarz criterion     :    5009.009

-----------------------------------------------------------------------------

In [100]:
print(ols_D_15.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3610
Adjusted R-squared  :      0.3577
Sum squared residual:     763.626                F-statistic           :    110.1959
Sigma-square        :       0.170                Prob(F-statistic)     :           0
S.E. of regression  :       0.413                Log likelihood        :   -2394.591
Sigma-square ML     :       0.169                Akaike info criterion :    4837.181
S.E of regression ML:      0.4115                Schwarz criterion     :    4991.119

-----------------------------------------------------------------------------

In [101]:
print(ols_D_16.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
R-squared           :      0.3576
Adjusted R-squared  :      0.3543
Sum squared residual:     767.692                F-statistic           :    108.5793
Sigma-square        :       0.171                Prob(F-statistic)     :           0
S.E. of regression  :       0.414                Log likelihood        :   -2406.565
Sigma-square ML     :       0.170                Akaike info criterion :    4861.130
S.E of regression ML:      0.4126                Schwarz criterion     :    5015.067

-----------------------------------------------------------------------------

Queen방식과 DistanceBand방식의 가중행렬을 사용한 모든 모형에서 공통적으로 Moran's I의 p-value < 0.05인 것을 확인 할 수 있다.  
즉 모든 모형에서 공간자기상관이 존재한다.

또한 모든 모형에서 공통적으로 Robust LM(lag) < Robust LM(error)인 것을 확인 할 수 있다. 따라서 SLM모형보다는 SEM모형이 적절하다.

## 공간회귀모형 적합

### SEM_ML모형 적합

In [102]:
#Queen방식의 공간가중행렬을 사용하여 ML방식으로 공간오차모형 적합

mlerror_Q_1 = ML_Error(y=y_1, x=x_1, w=W_Q, method='full', name_x=name_x_1, name_y=name_y_1)

mlerror_Q_2 = ML_Error(y=y_2, x=x_2, w=W_Q, method='full', name_x=name_x_2, name_y=name_y_2)

mlerror_Q_3 = ML_Error(y=y_3, x=x_3, w=W_Q, method='full', name_x=name_x_3, name_y=name_y_3)

mlerror_Q_4 = ML_Error(y=y_4, x=x_4, w=W_Q, method='full', name_x=name_x_4, name_y=name_y_4)

mlerror_Q_5 = ML_Error(y=y_5, x=x_5, w=W_Q, method='full', name_x=name_x_5, name_y=name_y_5)

mlerror_Q_6 = ML_Error(y=y_6, x=x_6, w=W_Q, method='full', name_x=name_x_6, name_y=name_y_6)

mlerror_Q_7 = ML_Error(y=y_7, x=x_7, w=W_Q, method='full', name_x=name_x_7, name_y=name_y_7)

mlerror_Q_8 = ML_Error(y=y_8, x=x_8, w=W_Q, method='full', name_x=name_x_8, name_y=name_y_8)

mlerror_Q_9 = ML_Error(y=y_9, x=x_9, w=W_Q, method='full', name_x=name_x_9, name_y=name_y_9)

mlerror_Q_10 = ML_Error(y=y_10, x=x_10, w=W_Q, method='full', name_x=name_x_10, name_y=name_y_10)

mlerror_Q_11 = ML_Error(y=y_11, x=x_11, w=W_Q, method='full', name_x=name_x_11, name_y=name_y_11)

mlerror_Q_12 = ML_Error(y=y_12, x=x_12, w=W_Q, method='full', name_x=name_x_12, name_y=name_y_12)

mlerror_Q_13 = ML_Error(y=y_13, x=x_13, w=W_Q, method='full', name_x=name_x_13, name_y=name_y_13)

mlerror_Q_14 = ML_Error(y=y_14, x=x_14, w=W_Q, method='full', name_x=name_x_14, name_y=name_y_14)

mlerror_Q_15 = ML_Error(y=y_15, x=x_15, w=W_Q, method='full', name_x=name_x_15, name_y=name_y_15)

mlerror_Q_16 = ML_Error(y=y_16, x=x_16, w=W_Q, method='full', name_x=name_x_16, name_y=name_y_16)

/opt/app-root/lib/python3.6/site-packages/scipy/optimize/_minimize.py:778: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)


In [103]:
#DistanceBand방식의 공간가중행렬을 사용하여 ML방식으로 공간오차모형 적합

mlerror_D_1 = ML_Error(y=y_1, x=x_1, w=W_D, method='full', name_x=name_x_1, name_y=name_y_1)

mlerror_D_2 = ML_Error(y=y_2, x=x_2, w=W_D, method='full', name_x=name_x_2, name_y=name_y_2)

mlerror_D_3 = ML_Error(y=y_3, x=x_3, w=W_D, method='full', name_x=name_x_3, name_y=name_y_3)

mlerror_D_4 = ML_Error(y=y_4, x=x_4, w=W_D, method='full', name_x=name_x_4, name_y=name_y_4)

mlerror_D_5 = ML_Error(y=y_5, x=x_5, w=W_D, method='full', name_x=name_x_5, name_y=name_y_5)

mlerror_D_6 = ML_Error(y=y_6, x=x_6, w=W_D, method='full', name_x=name_x_6, name_y=name_y_6)

mlerror_D_7 = ML_Error(y=y_7, x=x_7, w=W_D, method='full', name_x=name_x_7, name_y=name_y_7)

mlerror_D_8 = ML_Error(y=y_8, x=x_8, w=W_D, method='full', name_x=name_x_8, name_y=name_y_8)

mlerror_D_9 = ML_Error(y=y_9, x=x_9, w=W_D, method='full', name_x=name_x_9, name_y=name_y_9)

mlerror_D_10 = ML_Error(y=y_10, x=x_10, w=W_D, method='full', name_x=name_x_10, name_y=name_y_10)

mlerror_D_11 = ML_Error(y=y_11, x=x_11, w=W_D, method='full', name_x=name_x_11, name_y=name_y_11)

mlerror_D_12 = ML_Error(y=y_12, x=x_12, w=W_D, method='full', name_x=name_x_12, name_y=name_y_12)

mlerror_D_13 = ML_Error(y=y_13, x=x_13, w=W_D, method='full', name_x=name_x_13, name_y=name_y_13)

mlerror_D_14 = ML_Error(y=y_14, x=x_14, w=W_D, method='full', name_x=name_x_14, name_y=name_y_14)

mlerror_D_15 = ML_Error(y=y_15, x=x_15, w=W_D, method='full', name_x=name_x_15, name_y=name_y_15)

mlerror_D_16 = ML_Error(y=y_16, x=x_16, w=W_D, method='full', name_x=name_x_16, name_y=name_y_16)

/opt/app-root/lib/python3.6/site-packages/numpy/linalg/linalg.py:2159: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


### SEM_GM모형 적합

In [22]:
#Queen방식의 공간가중행렬을 사용하여 GM방식으로 공간오차모형 적합

gmerror_Q_1 = GM_Error(y=y_1, x=x_1, w=W_Q, name_x=name_x_1, name_y=name_y_1)

gmerror_Q_2 = GM_Error(y=y_2, x=x_2, w=W_Q, name_x=name_x_2, name_y=name_y_2)

gmerror_Q_3 = GM_Error(y=y_3, x=x_3, w=W_Q, name_x=name_x_3, name_y=name_y_3)

gmerror_Q_4 = GM_Error(y=y_4, x=x_4, w=W_Q, name_x=name_x_4, name_y=name_y_4)

gmerror_Q_5 = GM_Error(y=y_5, x=x_5, w=W_Q, name_x=name_x_5, name_y=name_y_5)

gmerror_Q_6 = GM_Error(y=y_6, x=x_6, w=W_Q, name_x=name_x_6, name_y=name_y_6)

gmerror_Q_7 = GM_Error(y=y_7, x=x_7, w=W_Q, name_x=name_x_7, name_y=name_y_7)

gmerror_Q_8 = GM_Error(y=y_8, x=x_8, w=W_Q, name_x=name_x_8, name_y=name_y_8)

gmerror_Q_9 = GM_Error(y=y_9, x=x_9, w=W_Q, name_x=name_x_9, name_y=name_y_9)

gmerror_Q_10 = GM_Error(y=y_10, x=x_10, w=W_Q, name_x=name_x_10, name_y=name_y_10)

gmerror_Q_11 = GM_Error(y=y_11, x=x_11, w=W_Q, name_x=name_x_11, name_y=name_y_11)

gmerror_Q_12 = GM_Error(y=y_12, x=x_12, w=W_Q, name_x=name_x_12, name_y=name_y_12)

gmerror_Q_13 = GM_Error(y=y_13, x=x_13, w=W_Q, name_x=name_x_13, name_y=name_y_13)

gmerror_Q_14 = GM_Error(y=y_14, x=x_14, w=W_Q, name_x=name_x_14, name_y=name_y_14)

gmerror_Q_15 = GM_Error(y=y_15, x=x_15, w=W_Q, name_x=name_x_15, name_y=name_y_15)

gmerror_Q_16 = GM_Error(y=y_16, x=x_16, w=W_Q, name_x=name_x_16, name_y=name_y_16)

NameError: name 'y_1' is not defined

In [105]:
#DistanceBand방식의 공간가중행렬을 사용하여 GM방식으로 공간오차모형 적합

gmerror_D_1 = GM_Error(y=y_1, x=x_1, w=W_D, name_x=name_x_1, name_y=name_y_1)

gmerror_D_2 = GM_Error(y=y_2, x=x_2, w=W_D, name_x=name_x_2, name_y=name_y_2)

gmerror_D_3 = GM_Error(y=y_3, x=x_3, w=W_D, name_x=name_x_3, name_y=name_y_3)

gmerror_D_4 = GM_Error(y=y_4, x=x_4, w=W_D, name_x=name_x_4, name_y=name_y_4)

gmerror_D_5 = GM_Error(y=y_5, x=x_5, w=W_D, name_x=name_x_5, name_y=name_y_5)

gmerror_D_6 = GM_Error(y=y_6, x=x_6, w=W_D, name_x=name_x_6, name_y=name_y_6)

gmerror_D_7 = GM_Error(y=y_7, x=x_7, w=W_D, name_x=name_x_7, name_y=name_y_7)

gmerror_D_8 = GM_Error(y=y_8, x=x_8, w=W_D, name_x=name_x_8, name_y=name_y_8)

gmerror_D_9 = GM_Error(y=y_9, x=x_9, w=W_D, name_x=name_x_9, name_y=name_y_9)

gmerror_D_10 = GM_Error(y=y_10, x=x_10, w=W_D, name_x=name_x_10, name_y=name_y_10)

gmerror_D_11 = GM_Error(y=y_11, x=x_11, w=W_D, name_x=name_x_11, name_y=name_y_11)

gmerror_D_12 = GM_Error(y=y_12, x=x_12, w=W_D, name_x=name_x_12, name_y=name_y_12)

gmerror_D_13 = GM_Error(y=y_13, x=x_13, w=W_D, name_x=name_x_13, name_y=name_y_13)

gmerror_D_14 = GM_Error(y=y_14, x=x_14, w=W_D, name_x=name_x_14, name_y=name_y_14)

gmerror_D_15 = GM_Error(y=y_15, x=x_15, w=W_D, name_x=name_x_15, name_y=name_y_15)

gmerror_D_16 = GM_Error(y=y_16, x=x_16, w=W_D, name_x=name_x_16, name_y=name_y_16)

## 공간회귀모형의 적합성 확인

### 각 공간회귀모형의 요약 정보 확인

#### Queen방식의 공간가중행렬을 사용한 SEM_ML 요약 정보

In [106]:
print(mlerror_Q_1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3518
Sigma-square ML     :       0.167                Log likelihood        :   -2385.851
S.E of regression   :       0.409                Akaike info criterion :    4819.702
                                                 Schwarz criterion     :    4973.640

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [107]:
print(mlerror_Q_2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3549
Sigma-square ML     :       0.167                Log likelihood        :   -2379.961
S.E of regression   :       0.409                Akaike info criterion :    4807.923
                                                 Schwarz criterion     :    4961.860

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [108]:
print(mlerror_Q_3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3587
Sigma-square ML     :       0.167                Log likelihood        :   -2371.266
S.E of regression   :       0.408                Akaike info criterion :    4790.532
                                                 Schwarz criterion     :    4944.470

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [109]:
print(mlerror_Q_4.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3518
Sigma-square ML     :       0.167                Log likelihood        :   -2385.851
S.E of regression   :       0.409                Akaike info criterion :    4819.702
                                                 Schwarz criterion     :    4973.640

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [110]:
print(mlerror_Q_5.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3559
Sigma-square ML     :       0.167                Log likelihood        :   -2374.854
S.E of regression   :       0.408                Akaike info criterion :    4797.709
                                                 Schwarz criterion     :    4951.646

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [111]:
print(mlerror_Q_6.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3575
Sigma-square ML     :       0.167                Log likelihood        :   -2372.354
S.E of regression   :       0.408                Akaike info criterion :    4792.709
                                                 Schwarz criterion     :    4946.646

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [112]:
print(mlerror_Q_7.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3610
Sigma-square ML     :       0.166                Log likelihood        :   -2364.505
S.E of regression   :       0.408                Akaike info criterion :    4777.010
                                                 Schwarz criterion     :    4930.947

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [113]:
print(mlerror_Q_8.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3559
Sigma-square ML     :       0.167                Log likelihood        :   -2374.854
S.E of regression   :       0.408                Akaike info criterion :    4797.709
                                                 Schwarz criterion     :    4951.646

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [114]:
print(mlerror_Q_9.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3554
Sigma-square ML     :       0.167                Log likelihood        :   -2379.811
S.E of regression   :       0.409                Akaike info criterion :    4807.621
                                                 Schwarz criterion     :    4961.558

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [115]:
print(mlerror_Q_10.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3568
Sigma-square ML     :       0.167                Log likelihood        :   -2376.738
S.E of regression   :       0.409                Akaike info criterion :    4801.476
                                                 Schwarz criterion     :    4955.413

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [116]:
print(mlerror_Q_11.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3589
Sigma-square ML     :       0.167                Log likelihood        :   -2371.180
S.E of regression   :       0.408                Akaike info criterion :    4790.360
                                                 Schwarz criterion     :    4944.297

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [117]:
print(mlerror_Q_12.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3554
Sigma-square ML     :       0.167                Log likelihood        :   -2379.811
S.E of regression   :       0.409                Akaike info criterion :    4807.621
                                                 Schwarz criterion     :    4961.558

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [118]:
print(mlerror_Q_13.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3569
Sigma-square ML     :       0.167                Log likelihood        :   -2374.694
S.E of regression   :       0.408                Akaike info criterion :    4797.387
                                                 Schwarz criterion     :    4951.325

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [19]:
print(mlerror_Q_14.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3578
Sigma-square ML     :       0.167                Log likelihood        :   -2373.559
S.E of regression   :       0.408                Akaike info criterion :    4795.119
                                                 Schwarz criterion     :    4949.056

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [120]:
print(mlerror_Q_15.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3604
Sigma-square ML     :       0.166                Log likelihood        :   -2367.540
S.E of regression   :       0.408                Akaike info criterion :    4783.079
                                                 Schwarz criterion     :    4937.017

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [121]:
print(mlerror_Q_16.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3569
Sigma-square ML     :       0.167                Log likelihood        :   -2374.694
S.E of regression   :       0.408                Akaike info criterion :    4797.387
                                                 Schwarz criterion     :    4951.325

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

#### DistanceBand방식의 공간가중행렬을 사용한 SEM_ML 요약 정보

In [122]:
print(mlerror_D_1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1468
Sigma-square ML     :  152286.145                Log likelihood        :         nan
S.E of regression   :     390.239                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [123]:
print(mlerror_D_2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1565
Sigma-square ML     :  150783.048                Log likelihood        :         nan
S.E of regression   :     388.308                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [124]:
print(mlerror_D_3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1636
Sigma-square ML     :  147433.570                Log likelihood        :         nan
S.E of regression   :     383.971                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [125]:
print(mlerror_D_4.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1468
Sigma-square ML     :  152286.145                Log likelihood        :         nan
S.E of regression   :     390.239                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [126]:
print(mlerror_D_5.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1494
Sigma-square ML     :  152020.478                Log likelihood        :         nan
S.E of regression   :     389.898                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [127]:
print(mlerror_D_6.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1592
Sigma-square ML     :  150488.785                Log likelihood        :         nan
S.E of regression   :     387.929                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [128]:
print(mlerror_D_7.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1659
Sigma-square ML     :  147157.122                Log likelihood        :         nan
S.E of regression   :     383.611                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [129]:
print(mlerror_D_8.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1494
Sigma-square ML     :  152020.478                Log likelihood        :         nan
S.E of regression   :     389.898                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [130]:
print(mlerror_D_9.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1538
Sigma-square ML     :  150364.976                Log likelihood        :         nan
S.E of regression   :     387.769                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [131]:
print(mlerror_D_10.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1618
Sigma-square ML     :  149114.870                Log likelihood        :         nan
S.E of regression   :     386.154                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [132]:
print(mlerror_D_11.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1659
Sigma-square ML     :  146612.261                Log likelihood        :         nan
S.E of regression   :     382.900                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [133]:
print(mlerror_D_12.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1538
Sigma-square ML     :  150364.976                Log likelihood        :         nan
S.E of regression   :     387.769                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [134]:
print(mlerror_D_13.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1508
Sigma-square ML     :  151579.043                Log likelihood        :         nan
S.E of regression   :     389.332                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [21]:
print(mlerror_D_14.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1601
Sigma-square ML     :  150112.184                Log likelihood        :         nan
S.E of regression   :     387.443                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [136]:
print(mlerror_D_15.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1658
Sigma-square ML     :  146995.243                Log likelihood        :         nan
S.E of regression   :     383.400                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

In [137]:
print(mlerror_D_16.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR (METHOD = FULL)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.1508
Sigma-square ML     :  151579.043                Log likelihood        :         nan
S.E of regression   :     389.332                Akaike info criterion :         nan
                                                 Schwarz criterion     :         nan

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------------------

#### Queen방식의 공간가중행렬을 사용한 SEM_GM 요약 정보

In [138]:
print(gmerror_Q_1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3519

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0142041       0.0141443      -1.0042241       0.3152706
               단속카메라      -0.1098368       0.0420721      -2.6106777       0.0090363
              도로안내표지       0.0505298       0.0136590       3.6993883       0.0002161
      

In [139]:
print(gmerror_Q_2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3550

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0169025       0.0140167      -1.2058815       0.2278632
               단속카메라      -0.1170691       0.0420670      -2.7829229       0.0053872
              도로안내표지       0.0508651       0.0136423       3.7284819       0.0001926
      

In [140]:
print(gmerror_Q_3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3588

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0277251       0.0140493      -1.9734142       0.0484484
               단속카메라      -0.1163839       0.0419851      -2.7720287       0.0055708
              도로안내표지       0.0521020       0.0136213       3.8250499       0.0001307
      

In [141]:
print(gmerror_Q_4.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3519

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0142041       0.0141443      -1.0042241       0.3152706
               단속카메라      -0.1098368       0.0420721      -2.6106777       0.0090363
              도로안내표지       0.0505298       0.0136590       3.6993883       0.0002161
      

In [142]:
print(gmerror_Q_5.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3560

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0144565       0.0140405      -1.0296273       0.3031850
               단속카메라      -0.1140571       0.0419957      -2.7159206       0.0066092
              도로안내표지       0.0492866       0.0136295       3.6161842       0.0002990
      

In [143]:
print(gmerror_Q_6.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3576

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0160935       0.0139558      -1.1531790       0.2488369
               단속카메라      -0.1192841       0.0420058      -2.8397033       0.0045156
              도로안내표지       0.0499157       0.0136205       3.6647544       0.0002476
      

In [144]:
print(gmerror_Q_7.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3611

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0258651       0.0140055      -1.8467800       0.0647790
               단속카메라      -0.1194805       0.0419321      -2.8493816       0.0043804
              도로안내표지       0.0510423       0.0136023       3.7524755       0.0001751
      

In [145]:
print(gmerror_Q_8.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3560

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0144565       0.0140405      -1.0296273       0.3031850
               단속카메라      -0.1140571       0.0419957      -2.7159206       0.0066092
              도로안내표지       0.0492866       0.0136295       3.6161842       0.0002990
      

In [146]:
print(gmerror_Q_9.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3555

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0201438       0.0140303      -1.4357351       0.1510777
               단속카메라      -0.1184500       0.0421277      -2.8116879       0.0049282
              도로안내표지       0.0509986       0.0136461       3.7372324       0.0001861
      

In [147]:
print(gmerror_Q_10.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3569

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0205619       0.0139530      -1.4736588       0.1405735
               단속카메라      -0.1219091       0.0421051      -2.8953559       0.0037873
              도로안내표지       0.0511693       0.0136347       3.7528731       0.0001748
      

In [148]:
print(gmerror_Q_11.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3590

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0283256       0.0140221      -2.0200648       0.0433767
               단속카메라      -0.1181701       0.0420441      -2.8106247       0.0049445
              도로안내표지       0.0519585       0.0136204       3.8147531       0.0001363
      

In [149]:
print(gmerror_Q_12.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3555

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0201438       0.0140303      -1.4357351       0.1510777
               단속카메라      -0.1184500       0.0421277      -2.8116879       0.0049282
              도로안내표지       0.0509986       0.0136461       3.7372324       0.0001861
      

In [150]:
print(gmerror_Q_13.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3570

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0180577       0.0139679      -1.2928035       0.1960790
               단속카메라      -0.1183983       0.0420465      -2.8158902       0.0048642
              도로안내표지       0.0504694       0.0136297       3.7029055       0.0002131
      

In [25]:
print(gmerror_Q_14.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3579

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0184037       0.0139003      -1.3239810       0.1855093
               단속카메라      -0.1223506       0.0420552      -2.9092866       0.0036225
              도로안내표지       0.0510014       0.0136243       3.7434245       0.0001815
      

In [152]:
print(gmerror_Q_15.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3605

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0268484       0.0139776      -1.9208180       0.0547547
               단속카메라      -0.1212581       0.0420005      -2.8870655       0.0038885
              도로안내표지       0.0519117       0.0136101       3.8142019       0.0001366
      

In [153]:
print(gmerror_Q_16.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3570

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0180577       0.0139679      -1.2928035       0.1960790
               단속카메라      -0.1183983       0.0420465      -2.8158902       0.0048642
              도로안내표지       0.0504694       0.0136297       3.7029055       0.0002131
      

#### DistanceBand방식의 공간가중행렬을 사용한 SEM_GM 요약 정보

In [154]:
print(gmerror_D_1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3528

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0107385       0.0121623      -0.8829318       0.3772731
               단속카메라      -0.1026551       0.0427261      -2.4026293       0.0162777
              도로안내표지       0.0537351       0.0137499       3.9080243       0.0000931
      

In [155]:
print(gmerror_D_2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3557

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0127858       0.0121387      -1.0533081       0.2921998
               단속카메라      -0.1122320       0.0426502      -2.6314510       0.0085021
              도로안내표지       0.0539842       0.0137158       3.9359160       0.0000829
      

In [156]:
print(gmerror_D_3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3594

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0227467       0.0122213      -1.8612344       0.0627111
               단속카메라      -0.1121339       0.0424970      -2.6386332       0.0083241
              도로안내표지       0.0551525       0.0136775       4.0323671       0.0000552
      

In [157]:
print(gmerror_D_4.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3528

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0107385       0.0121623      -0.8829318       0.3772731
               단속카메라      -0.1026551       0.0427261      -2.4026293       0.0162777
              도로안내표지       0.0537351       0.0137499       3.9080243       0.0000931
      

In [158]:
print(gmerror_D_5.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3567

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0109176       0.0121254      -0.9003891       0.3679132
               단속카메라      -0.1073696       0.0426059      -2.5200608       0.0117335
              도로안내표지       0.0517960       0.0137114       3.7775867       0.0001584
      

In [159]:
print(gmerror_D_6.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3582

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0122994       0.0121132      -1.0153710       0.3099290
               단속카메라      -0.1144588       0.0425653      -2.6890150       0.0071663
              도로안내표지       0.0524908       0.0136902       3.8341877       0.0001260
      

In [160]:
print(gmerror_D_7.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3616

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0213724       0.0122038      -1.7512946       0.0798952
               단속카메라      -0.1150258       0.0424245      -2.7113055       0.0067019
              도로안내표지       0.0536002       0.0136560       3.9250321       0.0000867
      

In [161]:
print(gmerror_D_8.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3567

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0109176       0.0121254      -0.9003891       0.3679132
               단속카메라      -0.1073696       0.0426059      -2.5200608       0.0117335
              도로안내표지       0.0517960       0.0137114       3.7775867       0.0001584
      

In [162]:
print(gmerror_D_9.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3562

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0167549       0.0121778      -1.3758554       0.1688664
               단속카메라      -0.1157054       0.0426914      -2.7102768       0.0067227
              도로안내표지       0.0537581       0.0137131       3.9202050       0.0000885
      

In [163]:
print(gmerror_D_10.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3576

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0169018       0.0121546      -1.3905687       0.1643562
               단속카메라      -0.1196404       0.0426355      -2.8061197       0.0050142
              도로안내표지       0.0539188       0.0136944       3.9372764       0.0000824
      

In [164]:
print(gmerror_D_11.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3596

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0236689       0.0122148      -1.9377331       0.0526558
               단속카메라      -0.1153401       0.0425472      -2.7108754       0.0067106
              도로안내표지       0.0547587       0.0136744       4.0044686       0.0000622
      

In [165]:
print(gmerror_D_12.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3562

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0167549       0.0121778      -1.3758554       0.1688664
               단속카메라      -0.1157054       0.0426914      -2.7102768       0.0067227
              도로안내표지       0.0537581       0.0137131       3.9202050       0.0000885
      

In [166]:
print(gmerror_D_13.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3576

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0140859       0.0121271      -1.1615185       0.2454311
               단속카메라      -0.1142207       0.0426112      -2.6805310       0.0073505
              도로안내표지       0.0530065       0.0136982       3.8695970       0.0001090
      

In [26]:
print(gmerror_D_14.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3585

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0143086       0.0121118      -1.1813786       0.2374524
               단속카메라      -0.1192311       0.0425869      -2.7997165       0.0051148
              도로안내표지       0.0536049       0.0136850       3.9170573       0.0000896
      

In [168]:
print(gmerror_D_15.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3610

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0221401       0.0122051      -1.8140027       0.0696773
               단속카메라      -0.1181831       0.0424825      -2.7819261       0.0054037
              도로안내표지       0.0545454       0.0136596       3.9931800       0.0000652
      

In [169]:
print(gmerror_D_16.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    ['사고횟수']                Number of Observations:        4510
Mean dependent var  :      0.1191                Number of Variables   :          24
S.D. dependent var  :      0.5148                Degrees of Freedom    :        4486
Pseudo R-squared    :      0.3576

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0140859       0.0121271      -1.1615185       0.2454311
               단속카메라      -0.1142207       0.0426112      -2.6805310       0.0073505
              도로안내표지       0.0530065       0.0136982       3.8695970       0.0001090
      

### ML방식 모형의 정규성 검정

In [170]:
#Queen방식의 공간가중행렬을 사용한 모형의 정규성 검정

stat_Q_1, p_Q_1 = shapiro(mlerror_Q_1.e_filtered)
stat_Q_2, p_Q_2 = shapiro(mlerror_Q_2.e_filtered)
stat_Q_3, p_Q_3 = shapiro(mlerror_Q_3.e_filtered)
stat_Q_4, p_Q_4 = shapiro(mlerror_Q_4.e_filtered)
stat_Q_5, p_Q_5 = shapiro(mlerror_Q_5.e_filtered)
stat_Q_6, p_Q_6 = shapiro(mlerror_Q_6.e_filtered)
stat_Q_7, p_Q_7 = shapiro(mlerror_Q_7.e_filtered)
stat_Q_8, p_Q_8 = shapiro(mlerror_Q_8.e_filtered)
stat_Q_9, p_Q_9 = shapiro(mlerror_Q_9.e_filtered)
stat_Q_10, p_Q_10 = shapiro(mlerror_Q_10.e_filtered)
stat_Q_11, p_Q_11 = shapiro(mlerror_Q_11.e_filtered)
stat_Q_12, p_Q_12 = shapiro(mlerror_Q_12.e_filtered)
stat_Q_13, p_Q_13 = shapiro(mlerror_Q_13.e_filtered)
stat_Q_14, p_Q_14 = shapiro(mlerror_Q_14.e_filtered)
stat_Q_15, p_Q_15 = shapiro(mlerror_Q_15.e_filtered)
stat_Q_16, p_Q_16 = shapiro(mlerror_Q_16.e_filtered)

In [171]:
print(stat_Q_1, p_Q_1)
print(stat_Q_2, p_Q_2)
print(stat_Q_3, p_Q_3)
print(stat_Q_4, p_Q_4)
print(stat_Q_5, p_Q_5)
print(stat_Q_6, p_Q_6)
print(stat_Q_7, p_Q_7)
print(stat_Q_8, p_Q_8)
print(stat_Q_9, p_Q_9)
print(stat_Q_10, p_Q_10) 
print(stat_Q_11, p_Q_11)
print(stat_Q_12, p_Q_12)
print(stat_Q_13, p_Q_13)
print(stat_Q_14, p_Q_14)
print(stat_Q_15, p_Q_15)
print(stat_Q_16, p_Q_16)

0.5762312412261963 0.0
0.5804355144500732 0.0
0.5897236466407776 0.0
0.5762312412261963 0.0
0.5824097990989685 0.0
0.5842154026031494 0.0
0.5924241542816162 0.0
0.5824097990989685 0.0
0.5832849740982056 0.0
0.5849684476852417 0.0
0.5905818939208984 0.0
0.5832849740982056 0.0
0.5845566987991333 0.0
0.5851827263832092 0.0
0.5923278331756592 0.0
0.5845566987991333 0.0


In [172]:
#DistanceBand방식의 공간가중행렬을 사용한 모형의 정규성 검정

stat_D_1, p_D_1 = shapiro(mlerror_D_1.e_filtered)
stat_D_2, p_D_2 = shapiro(mlerror_D_2.e_filtered)
stat_D_3, p_D_3 = shapiro(mlerror_D_3.e_filtered)
stat_D_4, p_D_4 = shapiro(mlerror_D_4.e_filtered)
stat_D_5, p_D_5 = shapiro(mlerror_D_5.e_filtered)
stat_D_6, p_D_6 = shapiro(mlerror_D_6.e_filtered)
stat_D_7, p_D_7 = shapiro(mlerror_D_7.e_filtered)
stat_D_8, p_D_8 = shapiro(mlerror_D_8.e_filtered)
stat_D_9, p_D_9 = shapiro(mlerror_D_9.e_filtered)
stat_D_10, p_D_10 = shapiro(mlerror_D_10.e_filtered)
stat_D_11, p_D_11 = shapiro(mlerror_D_11.e_filtered)
stat_D_12, p_D_12 = shapiro(mlerror_D_12.e_filtered)
stat_D_13, p_D_13 = shapiro(mlerror_D_13.e_filtered)
stat_D_14, p_D_14 = shapiro(mlerror_D_14.e_filtered)
stat_D_15, p_D_15 = shapiro(mlerror_D_15.e_filtered)
stat_D_16, p_D_16 = shapiro(mlerror_D_16.e_filtered)

In [173]:
print(stat_D_1, p_D_1)
print(stat_D_2, p_D_2)
print(stat_D_3, p_D_3)
print(stat_D_4, p_D_4)
print(stat_D_5, p_D_5)
print(stat_D_6, p_D_6)
print(stat_D_7, p_D_7)
print(stat_D_8, p_D_8)
print(stat_D_9, p_D_9)
print(stat_D_10, p_D_10) 
print(stat_D_11, p_D_11)
print(stat_D_12, p_D_12)
print(stat_D_13, p_D_13)
print(stat_D_14, p_D_14)
print(stat_D_15, p_D_15)
print(stat_D_16, p_D_16)

0.9528864026069641 7.70557783922496e-36
0.9532575011253357 9.980889787246468e-36
0.9564679861068726 1.0048473537770612e-34
0.9528864026069641 7.70557783922496e-36
0.9519246816635132 3.970056749755318e-36
0.952560305595398 6.146437833343409e-36
0.9566429853439331 1.1439632071882922e-34
0.9519246816635132 3.970056749755318e-36
0.9529136419296265 7.852803771397681e-36
0.9539515376091003 1.6265476905404045e-35
0.957687258720398 2.5021058472824727e-34
0.9529136419296265 7.852803771397681e-36
0.9516637325286865 3.32236574788908e-36
0.9487404823303223 4.7507816019969005e-37
0.9567775726318359 1.2643210865374025e-34
0.9516637325286865 3.32236574788908e-36


ML방식으로 공간오차모형을 추정할 때는 정규성이 성립해야 하지만, 결과를 보면 Queen방식과 DistanceBand방식의 가중행렬을 사용하여  
ML방식으로 추정한 모든 모형에서 정규성이 위배됨을 알 수 있다. 따라서 ML방식의 모형은 부적절하며, GM방식으로 추정한 모형이 적절하다.

GM방식으로 추정된 모형 중 DistanceBand방식의 가중행렬을 사용하고,  
초등학교_300m, 유치원_500m 변수를 사용한 모형의 R-squared가 가장 높다.